In [1]:
import pandas as pd
import pymongo
from sqlalchemy import create_engine

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [5]:
quotes_mdb=client.quotes_db
quotes=quotes_mdb.quotes

In [11]:
quotes_df=pd.DataFrame(list(quotes.find()))
quotes_df.head()

,_id,quote_text,tags,author_name,author_born,author_description
0,60529faac01f5c36c3f24da4,“The world as we have created it is a process ...,"[change, deep-thoughts, thinking, world]",Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
1,60529faac01f5c36c3f24da5,"“It is our choices, Harry, that show what we t...","[abilities, choices]",J.K. Rowling,"July 31, 1965 in Yate, South Gloucestershire, ...",\n See also: Robert GalbraithAlthough s...
2,60529faac01f5c36c3f24da6,“There are only two ways to live your life. On...,"[inspirational, life, live, miracle, miracles]",Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
3,60529faac01f5c36c3f24da7,"“The person, be it gentleman or lady, who has ...","[aliteracy, books, classic, humor]",Jane Austen,"December 16, 1775 in Steventon Rectory, Hampsh...",\n Jane Austen was an English novelist ...
4,60529faac01f5c36c3f24da8,"“Imperfection is beauty, madness is genius and...","[be-yourself, inspirational]",Marilyn Monroe,"June 01, 1926 in The United States",\n Marilyn Monroe (born Norma Jeane Mor...


In [10]:
quotes_table = quotes_df[['_id','author_name','quote_text']].copy()
quotes_table=quotes_table.rename(columns={'_id':'id','quote_text':'text'}).set_index('id')
quotes_table

,author_name,text
id,,
60529faac01f5c36c3f24da4,Albert Einstein,“The world as we have created it is a process ...
60529faac01f5c36c3f24da5,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
60529faac01f5c36c3f24da6,Albert Einstein,“There are only two ways to live your life. On...
60529faac01f5c36c3f24da7,Jane Austen,"“The person, be it gentleman or lady, who has ..."
60529faac01f5c36c3f24da8,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."
...,...,...
60529faac01f5c36c3f24e03,Harper Lee,“You never really understand a person until yo...
60529faac01f5c36c3f24e04,Madeleine L'Engle,“You have to write the book that wants to be w...
60529faac01f5c36c3f24e05,Mark Twain,“Never tell the truth to people who are not wo...


In [12]:
author_information_table = quotes_df[['author_name','author_born','author_description']].copy()
quotes_table=author_information_table.rename(columns={'author_name':'name','author_born':'born','author_description':'description'})
quotes_table

,name,born,description
0,Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
1,J.K. Rowling,"July 31, 1965 in Yate, South Gloucestershire, ...",\n See also: Robert GalbraithAlthough s...
2,Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
3,Jane Austen,"December 16, 1775 in Steventon Rectory, Hampsh...",\n Jane Austen was an English novelist ...
4,Marilyn Monroe,"June 01, 1926 in The United States",\n Marilyn Monroe (born Norma Jeane Mor...
...,...,...,...
95,Harper Lee,"April 28, 1926 in Monroeville, Alabama, The Un...","\n Harper Lee, known as Nelle, was born..."
96,Madeleine L'Engle,"November 29, 1918 in New York City, New York, ...",\n Madeleine L'Engle was an American wr...
97,Mark Twain,"November 30, 1835 in Florida, Missouri, The Un...","\n Samuel Langhorne Clemens, better kno..."
98,Dr. Seuss,"March 02, 1904 in Springfield, MA, The United ...",\n Theodor Seuss Geisel was born 2 Marc...


#### 